# Step 1: Data and Model Preparation

In [64]:
exp_name = 'exp_053'

In [65]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_2_rpn import GraphBatcher

In [67]:
import tensorflow

In [5]:
from src.tensorflow.models.vsp_model_002 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_02 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [6]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1294)
np.random.seed(1295)

In [82]:
len(gl.pred2subj)

108073

In [83]:
gl.pred2subj[0].shape

(25, 23)

In [89]:
np.stack((gl.pred2obj[0], gl.pred2subj[0])).shape

(2, 25, 23)

In [7]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_2.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [8]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'split_stanford.pkl'))
misc = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'misc_stanford_2.pkl'))
img_no_ent = misc['img_no_ent']
img_no_pred = misc['img_no_pred']
image_size = misc['img_size']

In [70]:
train_idx

array([48943, 61931, 34126, ..., 68223, 34039, 36485])

In [9]:
train_idx = np.asarray(list(set(train_idx) - set(img_no_ent)))
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [22]:
d = batcher.loader.get_gt_batch([0], pack = 1)

In [23]:
d.keys()

dict_keys(['image_id', 'ent_lbl', 'ent_box', 'pred_lbl', 'pred_roles', 'ent_emb', 'pred_emb', 'num_ent', 'num_pred'])

In [27]:
d['num_pred']

array([25], dtype=int32)

In [10]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'coords.pkl'))

In [11]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [74]:
hi['image_id'].dtype

dtype('int64')

In [56]:
gl.noun_emb_dict.shape

(151, 300)

In [60]:
gl.pred_emb_dict[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [12]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [14]:
#
print(batcher.subset_idx)
print(len(batcher.subset_idx))

print(gl.size)

[62262 12785 75192 ... 60423 35265  9882]
72538
108073


In [75]:
valhi = batcher_val.next_batch()

In [76]:
# max(batcher.subset_idx)
valhi.keys()

dict_keys(['image_id', 'ent_lbl', 'ent_box', 'pred_lbl', 'pred_roles', 'ent_emb', 'pred_emb', 'num_ent', 'num_pred', 'proposal_features', 'proposal_boxes'])

In [79]:
valhi['pred_roles'][0].shape

(2, 14, 22)

In [32]:
 hi = batcher.next_batch()
batcher.next_batch().keys()

dict_keys(['image_id', 'ent_lbl', 'ent_box', 'pred_lbl', 'pred_roles', 'ent_emb', 'pred_emb', 'num_ent', 'num_pred', 'proposal_features', 'proposal_boxes'])

In [61]:
for k, values in hi.items():
    print("k", k)
    print(values[0])

k image_id
2366968
k ent_lbl
[ 38  74 107 137 136 137 144 137 142   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
k ent_box
[[0.41308594 0.22727805 0.5078125  0.5132085 ]
 [0.02929688 0.4941465  0.04980469 0.6363786 ]
 [0.5078125  0.23900855 0.69628906 0.34458286]
 [0.         0.11877111 0.8720703  0.7888748 ]
 [0.8652344  0.         0.99609375 0.3753754 ]
 [0.7832031  0.3049925  0.99609375 0.6569069 ]
 [0.47558594 0.56452936 0.60253906 0.79620636]
 [0.11328125 0.1158385  0.8613281  0.8006053 ]
 [0.12988281 0.11877111 0.86035156 0.7918074 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.     

In [50]:
hi['ent_box'].shape

(32, 32, 4)

In [51]:
hi['ent_lbl'].shape

(32, 32)

In [53]:
hi['num_ent'].shape

(32,)

In [34]:
hi['ent_box'].shape

(32, 32, 4)

In [54]:
hi['ent_lbl'][0].shape

(32,)

In [47]:
hi['pred_lbl'][0].shape

(18,)

In [41]:
hi['pred_roles'].shape

(32, 2, 18, 32)

In [15]:
108073/32

3377.28125

In [14]:
tf.reset_default_graph()

In [62]:
hi['proposal_features']

array([[[1.02888048e-01, 0.00000000e+00, 1.53515311e-02, ...,
         0.00000000e+00, 4.98018973e-02, 2.50358403e-01],
        [1.23843504e-02, 1.09997541e-02, 6.88910857e-02, ...,
         1.50139844e+00, 1.47067923e-02, 2.88907662e-02],
        [4.93718475e-01, 0.00000000e+00, 1.42769724e-01, ...,
         9.45440084e-02, 7.92656243e-02, 4.48529482e-01],
        ...,
        [4.30669747e-02, 0.00000000e+00, 9.01341438e-02, ...,
         1.41820514e+00, 5.95145002e-02, 2.48852260e-02],
        [7.12050823e-05, 2.64792368e-02, 5.08873314e-02, ...,
         0.00000000e+00, 1.96236432e-01, 3.13458920e-01],
        [2.71464046e-02, 5.70288766e-03, 5.18503040e-03, ...,
         2.13391948e+00, 1.03772908e-01, 2.43152529e-01]],

       [[0.00000000e+00, 1.30004418e+00, 2.63716459e-01, ...,
         7.70304352e-02, 5.42689681e-01, 3.80267262e-01],
        [8.56112167e-02, 5.63037276e-01, 6.63065314e-01, ...,
         9.29591238e-01, 8.17425787e-01, 3.13118696e+00],
        [0.00000000e+00, 

In [22]:
type(hi['proposal_features'][0])

numpy.ndarray

In [63]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=3,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
entity_nodes_emb Tensor("graph_generator/mp_iter_0/mp_output_heads/Reshape_1:0", shape=(?, ?, 300), dtype=float32)
emb_head_ent_cls <tf.Variable 'graph_generator/mlp_bank/emb_head_ent_cls:0' shape=(151, 300) dtype=float32_ref>
entity_nodes_emb Tensor("graph_generator/mp_iter_1/mp_output_heads/Reshape_1:0", shape=(?, ?, 300), dtype=float32)
emb_head_ent_cls <tf.Variable 'graph_generator/mlp_bank/emb_head_ent_cls:0' shape=(151, 300) dtype=float32_ref>
entity_nodes_emb Tensor("graph_generator/mp_iter_2/mp_output_heads/Reshape_1:0", shape=(?, ?, 300), dtype=float32)
emb_head_ent_cls <tf.Variable 'graph_generator/mlp_bank/e

In [16]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training


In [ ]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [ ]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [ ]:
trainer.train (
    max_steps=20000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


../../src/eval/graph_eval_3.py:95: RuntimeWarning: invalid value encountered in true_divide
  per_image_pred_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))
../../src/eval/graph_eval_3.py:219: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))


Step 1000: loss = 3.572000026702881 (2621.978 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.217, 'PerImagePredMR': 0.978, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.028, 'PerImageOneHopMP_at50_iou_0.5': 0.003, 'PerImageOneHopMR_at50_iou_0.5': 0.023}
Validation Results: {'PerImagePredMP': 0.215, 'PerImagePredMR': 0.97, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.03, 'PerImageOneHopMP_at50_iou_0.5': 0.004, 'PerImageOneHopMR_at50_iou_0.5': 0.027}
Step 2000: loss = 3.3929998874664307 (5140.357 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.262, 'PerImagePredMR': 0.962, 'PerImageOneHopMP_at100_iou_0.5': 0.003, 'PerImageOneHopMR_at100_iou_0.5': 0.033, 'PerImageOneHopMP_at50_iou_0.5': 0.005, 'PerImageOneHopMR_at50_iou_0.5': 0.028}
Validation Results: {'PerImagePredMP': 0.249, 'PerImagePredMR': 0.949, 'PerImageOneHopMP_at100_iou_0.5': 0.002, 'PerImageOneHopMR_at100_iou_0.5': 0.03, 'PerImageOneHopMP_

In [ ]:
trainer.train (
    max_steps=30000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


In [28]:
trainer.train (
    max_steps=10000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 0.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 44000: loss = 1.6820000410079956 (2402.352 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.325, 'PerImagePredMR': 0.965, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.053, 'PerImageOneHopMP_at50_iou_0.5': 0.007, 'PerImageOneHopMR_at50_iou_0.5': 0.045}
Validation Results: {'PerImagePredMP': 0.307, 'PerImagePredMR': 0.953, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.059, 'PerImageOneHopMP_at50_iou_0.5': 0.007, 'PerImageOneHopMR_at50_iou_0.5': 0.051}
Step 45000: loss = 2.1549999713897705 (4778.309 sec)
Training Batch Evaluation Results: {'PerImagePredMP': 0.317, 'PerImagePredMR': 0.969, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.052, 'PerImageOneHopMP_at50_iou_0.5': 0.006, 'PerImageOneHopMR_at50_iou_0.5': 0.044}
Validation Results: {'PerImagePredMP': 0.305, 'PerImagePredMR': 0.952, 'PerImageOneHopMP_at100_iou_0.5': 0.004, 'PerImageOneHopMR_at100_iou_0.5': 0.058, 'PerImageOne

KeyboardInterrupt: 

In [ ]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [24]:
# load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-48000'))
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-10000'))

INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_053/ckpt-10000


In [26]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_with_duplicates.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [27]:
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})


In [28]:
test_idx = np.asarray(list(set(test_idx) - set(img_no_ent)))

In [29]:
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [30]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
Batch 1/997 done in 1 seconds. Spent time for read=0.21 (avg 0.21), processing=1.60 (avg 1.60), write=0.00 (avg 0.00) seconds.
Batch 101/997 done in 9 seconds. Spent time for read=0.02 (avg 0.02), processing=0.07 (avg 0.08), write=0.00 (avg 0.00) seconds.
Batch 201/997 done in 18 seconds. Spent time for read=0.02 (avg 0.02), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 301/997 done in 26 seconds. Spent time for read=0.02 (avg 0.02), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 401/997 done in 34 seconds. Spent time for read=0.02 (avg 0.02), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 501/997 done in 43 seconds. Spent time for read=0.01 (avg 0.02), processing=0.07 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 601/997 done in 51 seconds. Spent time for read=0.02 (avg 0.02), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 701/997 done in 60 seconds. Spent time for read=0.01 (avg 0.02), processing=0.

In [31]:
test_results = test_evaluator.evaluate()
test_results

../../src/eval/graph_eval_3.py:95: RuntimeWarning: invalid value encountered in true_divide
  per_image_pred_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))
../../src/eval/graph_eval_3.py:219: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))


{'PerImagePredMP': 0.3005916677123855,
 'PerImagePredMR': 0.9516072689572087,
 'PerImageOneHopMP_at100_iou_0.5': 0.0022945162504705735,
 'PerImageOneHopMR_at100_iou_0.5': 0.03029896586794681,
 'PerImageOneHopMP_at50_iou_0.5': 0.0037181578617141425,
 'PerImageOneHopMR_at50_iou_0.5': 0.02496890873579256}

In [32]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-48000'))


INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_053/ckpt-48000


In [33]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_with_duplicates.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': 20,
    'filter_nonoverlap': True
})
test_idx = np.asarray(list(set(test_idx) - set(img_no_ent)))
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_proposals', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)
test_results = test_evaluator.evaluate()
test_results

Deploying
Batch 1/997 done in 0 seconds. Spent time for read=0.01 (avg 0.01), processing=0.09 (avg 0.09), write=0.00 (avg 0.00) seconds.
Batch 101/997 done in 7 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.06), write=0.00 (avg 0.00) seconds.
Batch 201/997 done in 14 seconds. Spent time for read=0.01 (avg 0.01), processing=0.06 (avg 0.06), write=0.00 (avg 0.00) seconds.
Batch 301/997 done in 21 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.06), write=0.00 (avg 0.00) seconds.
Batch 401/997 done in 28 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.06), write=0.00 (avg 0.00) seconds.
Batch 501/997 done in 35 seconds. Spent time for read=0.00 (avg 0.01), processing=0.07 (avg 0.06), write=0.00 (avg 0.00) seconds.
Batch 601/997 done in 42 seconds. Spent time for read=0.00 (avg 0.00), processing=0.07 (avg 0.06), write=0.00 (avg 0.00) seconds.
Batch 701/997 done in 49 seconds. Spent time for read=0.01 (avg 0.00), processing=0.

{'PerImagePredMP': 0.31596530304931614,
 'PerImagePredMR': 0.9638242496028098,
 'PerImageOneHopMP_at100_iou_0.5': 0.0038787802735600454,
 'PerImageOneHopMR_at100_iou_0.5': 0.053772982299779296,
 'PerImageOneHopMP_at50_iou_0.5': 0.0064173672982808395,
 'PerImageOneHopMR_at50_iou_0.5': 0.04664551777924036}

In [35]:
test_evaluator.args.keys()

dict_keys(['graph_loader', 'proposals', 'num_proposals', 'filter_nonoverlap'])

In [37]:
model.eval_ops

{'image_id': <tf.Tensor 'image_id:0' shape=(?,) dtype=int32>,
 'out_ent_prob': <tf.Tensor 'graph_generator/mp_iter_2/mp_output_heads/Softmax:0' shape=(?, ?, 151) dtype=float32>,
 'out_pred_prob': <tf.Tensor 'graph_generator/mp_iter_2/mp_output_heads/Softmax_1:0' shape=(?, 100, 51) dtype=float32>,
 'out_pred_roles': <tf.Tensor 'graph_generator/mp_iter_2/mp_update_role_edges/mul_5:0' shape=(?, 2, 100, ?) dtype=float32>,
 'out_pred_conf': <tf.Tensor 'graph_generator/mp_iter_2/mp_output_heads/Sigmoid_1:0' shape=(?, 100) dtype=float32>,
 'out_num_ent': <tf.Tensor 'objective/Fill:0' shape=(?,) dtype=int32>,
 'out_num_pred': <tf.Tensor 'objective/Fill_1:0' shape=(?,) dtype=int32>}

In [39]:
test_evaluator.data.keys()

dict_keys(['image_id', 'out_num_ent', 'out_num_pred', 'out_ent_prob', 'out_pred_prob', 'out_pred_roles', 'out_pred_conf'])

In [42]:
d = test_evaluator.data

In [45]:
len(d['image_id'])

31876

In [55]:
print(len(d['out_num_ent']))#20
print(len(d['out_num_pred'])) #100
print(len(d['out_ent_prob']))
print(d['out_ent_prob'][0].shape)

31876
31876
31876
(20, 151)


In [57]:
print(len(d['out_pred_prob']))
print(d['out_pred_prob'][0].shape)
print(len(d['out_pred_roles']))
print(d['out_pred_roles'][0].shape)
print(len(d['out_pred_conf']))
print(d['out_pred_conf'][0].shape)

31876
(100, 51)
31876
(2, 100, 20)
31876
(100,)


In [61]:
# d

In [60]:
# d['out_pred_roles']

In [63]:
d['out_ent_prob']

[array([[8.1022966e-12, 4.3446344e-06, 1.6611095e-05, ..., 5.6548789e-03,
         1.1335597e-02, 5.1691943e-05],
        [2.4679336e-12, 1.1198026e-06, 7.6645760e-05, ..., 5.7096058e-03,
         3.1565859e-03, 4.6727848e-07],
        [1.5250977e-13, 6.8888244e-06, 9.3215740e-06, ..., 6.8896356e-05,
         8.3047263e-07, 5.8320456e-06],
        ...,
        [1.0510156e-12, 6.3673051e-06, 2.3853660e-04, ..., 1.2402692e-02,
         1.9819615e-02, 6.3228686e-07],
        [1.2023595e-10, 4.0925117e-09, 4.2223201e-06, ..., 6.4577223e-05,
         1.9218381e-04, 9.0275535e-07],
        [4.2373813e-12, 1.3602680e-06, 3.1342715e-04, ..., 1.4396394e-02,
         4.2040460e-03, 3.4995080e-06]], dtype=float32),
 array([[8.7690932e-15, 7.5297470e-07, 5.7762222e-06, ..., 1.8004510e-05,
         6.2695999e-06, 4.9815767e-06],
        [5.1862486e-12, 7.4461027e-06, 1.4112040e-05, ..., 1.1842903e-03,
         2.7759746e-03, 4.1135877e-06],
        [4.2479528e-10, 1.7702523e-05, 2.0131512e-05, ...,